In [3]:
from neo4j.v1 import GraphDatabase
import numpy as np

In [2]:
driver = GraphDatabase.driver('bolt://localhost:7687',auth=('neo4j', 'password'))


In [4]:
def print_results(array):
    a = np.array(array)
    print('Average response: {}ms'.format(a.mean()))
    print('Max response: {}ms'.format(max(a)))
    print('Min response: {}ms'.format(min(a)))
    for p in [25,50,75,90,95,99]:
        print('{}th percentile: {}ms'.format(p, np.percentile(a,p)))
        
def test_query(query):
    session = driver.session()
    results = []
    for _ in range(1000):
        r = session.run(query)
        summary = r.summary()
        millis = summary.result_available_after + summary.result_consumed_after
        results.append(millis)
    session.close()
    print_results(results)
    
def import_query(query):
    with driver.session() as session:
        session.run(query)

def delete_db_query():
    delete_query = """
    CALL apoc.periodic.commit('MATCH (n) WITH n limit $limit 
                           DETACH DELETE n RETURN count(*)', 
                          {limit:100000})
    """
    with driver.session() as session:
        session.run(delete_query)

# First example with reaction type as attribute

## Import

In [6]:
import_as_attribute = """

CALL apoc.periodic.iterate(
"UNWIND range(0,100) as i
return i",
"CREATE (post:Post{id:i})
WITH i,post
UNWIND range (0,10000) as j
WITH i,j,post, CASE j%6 WHEN 0 THEN 'like'
WHEN 1 THEN 'love'
WHEN 2 THEN 'haha'
WHEN 3 THEN 'wow'
WHEN 4 THEN 'sad'
WHEN 5 THEN 'angry' END as reaction
CREATE (post)<-[:REACTION{type:reaction}]-(:User)",
{parallel:True})

"""

import_query(import_as_attribute)

## Load test

In [9]:
attribute_first_query = """
MATCH (p:Post)<-[rel]-()
RETURN p.id as id, rel.type as type, count(*) AS count
ORDER by count DESC limit 5
"""
test_query(attribute_first_query)

Average response: 993.477ms
Max response: 2046ms
Min response: 801ms
25th percentile: 848.0ms
50th percentile: 936.0ms
75th percentile: 1073.0ms
90th percentile: 1238.3000000000002ms
95th percentile: 1352.05ms
99th percentile: 1696.02ms


In [10]:
attribute_second_query = """
MATCH (p:Post)<-[rel]-()
WITH p, rel.type as type, count(*) AS count
ORDER by count DESC limit 5
RETURN p.id,type,count

"""
test_query(attribute_second_query)

Average response: 753.792ms
Max response: 1493ms
Min response: 677ms
25th percentile: 694.0ms
50th percentile: 711.0ms
75th percentile: 769.25ms
90th percentile: 874.2ms
95th percentile: 966.05ms
99th percentile: 1125.1ms


# Second example with reaction type as node label

In [12]:
delete_db_query()

## Import

In [14]:
import_as_node_label = """
CALL apoc.periodic.iterate(
"UNWIND range(0,100) as i
return i",
"CREATE (post:Post{id:i})
with i,post
UNWIND range(0,10000) as j
WITH i,post, CASE j%6 WHEN 0 THEN 'like'
WHEN 1 THEN 'love'
WHEN 2 THEN 'haha'
WHEN 3 THEN 'wow'
WHEN 4 THEN 'sad'
WHEN 5 THEN 'angry' END as reaction
// Create node with a dynamic label
call apoc.create.node([reaction], {}) yield node
CREATE (post)<-[:TO]-(node)
CREATE (node)<-[:REACTION]-(:User)",
{parallel:True})
"""
import_query(import_as_node_label)

## Load test

In [16]:
second_query = """
MATCH (p:Post)<--(reaction)
WITH p,labels(reaction) as labels,count(*) as count
ORDER BY count DESC LIMIT 5
RETURN p.id as id,labels[0] as type,count(*)

"""
test_query(second_query)

Average response: 659.676ms
Max response: 1092ms
Min response: 596ms
25th percentile: 607.0ms
50th percentile: 628.0ms
75th percentile: 675.25ms
90th percentile: 756.2ms
95th percentile: 843.0ms
99th percentile: 1029.06ms


# Third example with reaction type as relationship type

In [17]:
delete_db_query()

## Import

In [18]:
import_type_as_rel_type = """
CALL apoc.periodic.iterate(
"UNWIND range(0,100) as i
return i",
"CREATE (post:Post{id:i})
with i,post
UNWIND range(0,10000) as j
CREATE (u:User)
WITH i,post,u, CASE j%6 WHEN 0 THEN 'like'
WHEN 1 THEN 'love'
WHEN 2 THEN 'haha'
WHEN 3 THEN 'wow'
WHEN 4 THEN 'sad'
WHEN 5 THEN 'angry' END as reaction
// Create relationship with a dynamic type
CALL apoc.create.relationship(u, reaction, {}, post) YIELD rel
RETURN rel",
{parallel:True})
"""
import_query(import_type_as_rel_type)

## Load test

In [19]:
rel_type_first_query = """
MATCH (p:Post)<-[rel]-()
WITH p,type(rel) as type,count(*) as count
ORDER BY count DESC limit 5 
RETURN p.id as id,type,count

"""
test_query(rel_type_first_query)

Average response: 712.257ms
Max response: 927ms
Min response: 699ms
25th percentile: 705.0ms
50th percentile: 708.0ms
75th percentile: 712.0ms
90th percentile: 723.0ms
95th percentile: 742.05ms
99th percentile: 777.0ms


In [20]:
rel_type_second_query = """
MATCH (p:Post)
WITH p
UNWIND [{key:'like',count:size((p)<-[:like]-())},
        {key:'love',count:size((p)<-[:love]-())},
        {key:'haha',count:size((p)<-[:haha]-())},
        {key:'wow', count: size((p)<-[:wow]-())},
        {key:'sad', count: size((p)<-[:sad]-())},
        {key:'angry',count: size((p)<-[:angry]-())}] as k
WITH p,k.key as key,k.count as count
ORDER BY count DESC LIMIT 5
RETURN p.id as id,key,count

"""
test_query(rel_type_second_query)

Average response: 1.085ms
Max response: 93ms
Min response: 0ms
25th percentile: 1.0ms
50th percentile: 1.0ms
75th percentile: 1.0ms
90th percentile: 2.0ms
95th percentile: 2.0ms
99th percentile: 2.0ms


In [21]:
rel_type_apoc_query = """

MATCH (p:Post)
UNWIND apoc.node.relationship.types(p) as type
WITH p, type, apoc.node.degree.in(p, type) as count
ORDER BY count DESC LIMIT 5
RETURN p.id as post, type, count

"""
test_query(rel_type_apoc_query)

Average response: 1.969ms
Max response: 63ms
Min response: 1ms
25th percentile: 1.0ms
50th percentile: 2.0ms
75th percentile: 2.0ms
90th percentile: 3.0ms
95th percentile: 3.0ms
99th percentile: 4.0ms
